In [1]:
import warnings
warnings.simplefilter('ignore')

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv('raw_data/answer_train.csv')

print(train.shape)
train.head()

(60000, 2)


,filename,label
0,train/市管宗教活动场所.csv,文化休闲
1,train/南涧彝族自治县市汶上县残疾人联合会_残疾人证办理_.csv,医疗卫生
2,train/价格监测信息公开事项汇总信息_.xls,经济管理
3,train/县级公共图书馆文化馆总分馆制信息_大安市.csv,文化休闲
4,train/县级公共图书馆文化馆总分馆制信息_陇南市武都区.csv,文化休闲


In [3]:
test = pd.read_csv('raw_data/submit_example_test1.csv')

print(test.shape)
test.head()

(8000, 2)


,filename,label
0,test1/阳谷县司法所组织信息_.xls,工业
1,test1/济宁市辽宁省朝阳市龙城区县残疾人联合会_残疾人证办理_.csv,城乡建设
2,test1/企业变更信息_.xls,城乡建设
3,test1/设立经营性互联网文化单位审批_万州区.csv,生态环境
4,test1/事业企业“双随机一公开”监察检查记录_.xls,城乡建设


In [4]:
df_label = pd.DataFrame({'label': train.label.value_counts(normalize=True).index.tolist(),
                         'label_n': [i for i in range(train.label.nunique())]})

df_label.head(20)

,label,label_n
0,工业,0
1,文化休闲,1
2,教育科技,2
3,医疗卫生,3
4,文秘行政,4
5,生态环境,5
6,城乡建设,6
7,农业畜牧业,7
8,经济管理,8
9,交通运输,9


In [5]:
train = train.merge(df_label, on='label', how='left')

print(train.shape)
train.head()

(60000, 3)


,filename,label,label_n
0,train/市管宗教活动场所.csv,文化休闲,1
1,train/南涧彝族自治县市汶上县残疾人联合会_残疾人证办理_.csv,医疗卫生,3
2,train/价格监测信息公开事项汇总信息_.xls,经济管理,8
3,train/县级公共图书馆文化馆总分馆制信息_大安市.csv,文化休闲,1
4,train/县级公共图书馆文化馆总分馆制信息_陇南市武都区.csv,文化休闲,1


In [6]:
def clean_text(x):
    text = x.replace('train/', '').replace('.xls', '').replace('.csv', '').replace('_', ' ')
    return text

train['text'] = train['filename'].apply(lambda x: clean_text(x))
test['text'] = test['filename'].apply(lambda x: clean_text(x))

In [7]:
train.head()

,filename,label,label_n,text
0,train/市管宗教活动场所.csv,文化休闲,1,市管宗教活动场所
1,train/南涧彝族自治县市汶上县残疾人联合会_残疾人证办理_.csv,医疗卫生,3,南涧彝族自治县市汶上县残疾人联合会 残疾人证办理
2,train/价格监测信息公开事项汇总信息_.xls,经济管理,8,价格监测信息公开事项汇总信息
3,train/县级公共图书馆文化馆总分馆制信息_大安市.csv,文化休闲,1,县级公共图书馆文化馆总分馆制信息 大安市
4,train/县级公共图书馆文化馆总分馆制信息_陇南市武都区.csv,文化休闲,1,县级公共图书馆文化馆总分馆制信息 陇南市武都区


In [8]:
train_df = train[['text', 'label_n']]
train_df.columns = ['text', 'label']

print(train_df.shape)
train_df.head()

(60000, 2)


,text,label
0,市管宗教活动场所,1
1,南涧彝族自治县市汶上县残疾人联合会 残疾人证办理,3
2,价格监测信息公开事项汇总信息,8
3,县级公共图书馆文化馆总分馆制信息 大安市,1
4,县级公共图书馆文化馆总分馆制信息 陇南市武都区,1


In [9]:
train_df = train_df.sample(frac=1., random_state=1024)

eval_df = train_df[54000:]
train_df = train_df[:54000]

In [10]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [11]:
model_args = ClassificationArgs()

model_args.max_seq_length = 128
model_args.train_batch_size = 16
model_args.num_train_epochs = 3
model_args.fp16 = False
model_args.evaluate_during_training = True
model_args.overwrite_output_dir = True

In [12]:
model = ClassificationModel("bert", 
                            "hfl/chinese-roberta-wwm-ext",
                            num_labels=len(df_label),
                            args=model_args)

Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model che

In [13]:
model.train_model(train_df, eval_df=eval_df)

(10125,
 {'global_step': [2000, 3375, 4000, 6000, 6750, 8000, 10000, 10125],
  'mcc': [0.9476937708528343,
   0.9527763533978135,
   0.9577338457403467,
   0.9677364423372926,
   0.9699235225996395,
   0.9722837477080647,
   0.9746596788441761,
   0.9748424057549449],
  'train_loss': [0.15783660113811493,
   1.1505966186523438,
   0.01191699504852295,
   0.0014483332633972168,
   0.001240551471710205,
   0.11167295277118683,
   0.00037664175033569336,
   0.0004429817199707031],
  'eval_loss': [0.2014222281773885,
   0.1762613417506218,
   0.1625722239414851,
   0.12617032970984776,
   0.11873192139466604,
   0.11979165108998617,
   0.10948107000192006,
   0.10929350209236145]})

In [19]:
from sklearn.metrics import accuracy_score

result, _, _ = model.eval_model(eval_df, acc=accuracy_score)
result

{'mcc': 0.9748424057549449, 'acc': 0.977, 'eval_loss': 0.10929350209236145}

In [23]:
data = []
for i, row in test.iterrows():
    data.append(row['text'])

In [24]:
predictions, raw_outputs = model.predict(data)

In [30]:
sub = pd.read_csv('raw_data/submit_example_test1.csv')[['filename']]
sub['label_n'] = predictions
sub = pd.merge(sub, df_label, on='label_n', how='left')
sub.drop(['label_n'], axis=1, inplace=True)

print(sub.shape)
sub.head(10)

(8000, 2)


,filename,label
0,test1/阳谷县司法所组织信息_.xls,政法监察
1,test1/济宁市辽宁省朝阳市龙城区县残疾人联合会_残疾人证办理_.csv,医疗卫生
2,test1/企业变更信息_.xls,工业
3,test1/设立经营性互联网文化单位审批_万州区.csv,文化休闲
4,test1/事业企业“双随机一公开”监察检查记录_.xls,劳动人事
5,test1/昭阳区市广东省名牌产品工业类称号信息_.xls,工业
6,test1/市级民办职业培训学校设立审批_.xls,教育科技
7,test1/分县市、区建筑业生产状况_.xls,城乡建设
8,test1/辽宁省阜新市彰武县市国控污染物自动监控信息_.xls,生态环境
9,test1/水环境重点排污公司名单目录信息_.xls,生态环境


In [31]:
sub.to_csv('baseline_202011121514.csv', index=False)